 # Competición Sales Prediction con keras


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

os.chdir('drive/My Drive/')

 ## importar librerias y datos

In [3]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import TimeseriesGenerator
# from sklearn.model_selection import train_test_split

In [4]:
item_categories = pd.read_csv('/content/drive/MyDrive/item_categories.csv')
items = pd.read_csv('/content/drive/MyDrive/items.csv')
sales_train = pd.read_csv('/content/drive/MyDrive/sales_train.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')
shops = pd.read_csv('/content/drive/MyDrive/shops.csv')
test = pd.read_csv('/content/drive/MyDrive/test.csv')

datasets = {
    'item_categories': item_categories.columns.values,
    'items': items.columns.values,
    'sales_train': sales_train.columns.values,
    'sample_submission': sample_submission.columns.values,
    'shops': shops.columns.values,
    'test': test.columns.values
}

# Si cada dataset tiene menos de 6 columnas, completar con NaN.
for dataset in datasets:
    while len(datasets[dataset]) < 6:
        datasets[dataset] = np.append(datasets[dataset], '-')

pd.DataFrame(datasets)

,item_categories,items,sales_train,sample_submission,shops,test
0,item_category_name,item_name,date,ID,shop_name,ID
1,item_category_id,item_id,date_block_num,item_cnt_month,shop_id,shop_id
2,-,item_category_id,shop_id,-,-,item_id
3,-,-,item_id,-,-,-
4,-,-,item_price,-,-,-
5,-,-,item_cnt_day,-,-,-


In [5]:
# Transformar el tipo de datos de la columna 'shop_id' a string.
ids = test['shop_id'].astype(str)
# Unir una columna con el valor de la columna 'shop_id' y la columna 'item_id'
ids = ids.str.cat(test['item_id'].astype(str), sep='_')
print(ids.unique().shape)

(214200,)


In [8]:
ds = sales_train

ds.rename(columns={'item_cnt_day':'sales', 'date_block_num':'block'}, inplace=True)
w = ds.groupby(['block','shop_id','item_id']).sum()['sales'].reset_index()
v = ds.groupby(['block','shop_id','item_id']).mean()['item_price'].reset_index()
v['sales'] = w.sales
v = v.merge(test, on=['shop_id','item_id'], how='inner')

v = v.loc[v['sales'] != 0]
ds = v.sort_values(by=['item_id', 'shop_id', 'block'])

In [9]:
test_zero = np.setxor1d(test['ID'].values, v['ID'].values)
test_zero.shape

(102838,)

In [10]:
ds_pivoted = ds.pivot_table(index = ['ID'], values = ['sales'], columns = ['block'], fill_value = 0, aggfunc='sum').reset_index()
ds_pivoted.head()

ID sales                          ...                              
block        0  1  2  3  4  5  6  7  8  ... 24 25 26 27 28 29 30 31 32 33
0      0     0  0  0  0  0  0  0  0  0  ...  2  0  0  0  1  1  1  3  1  0
1      2     0  0  0  0  0  0  0  0  0  ...  0  0  0  0  3  2  0  1  3  1
2      3     0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  1  0  0
3      5     0  0  0  0  0  0  0  0  0  ...  1  0  0  0  3  0  0  0  1  1
4      6     0  0  0  0  0  0  0  0  0  ...  0  0  0  0  0  0  0  0  3  2

[5 rows x 35 columns]

In [11]:
shape_0 = ds.shape
for i in range(34):
    a = ds_pivoted.loc[ds_pivoted['sales'][i]==0]
    ds = pd.concat([ds, pd.DataFrame({'ID':a['ID'], 'block':i, 'sales':0})], ignore_index=True)

shape_1 = ds.shape
print(shape_0, shape_1)
ds = ds.sort_values(by=['ID', 'block'])

(599246, 6) (3786308, 6)


In [13]:
_ds = ds.drop(['shop_id','item_id','item_price','ID','block'],axis=1)
_ds.head()

,sales
599246,0.0
702790,0.0
805943,0.0
907571,0.0
1010276,0.0


In [15]:
_ds.shape

(3786308, 1)

In [21]:
from tensorflow.keras.utils import timeseries_dataset_from_array

X = timeseries_dataset_from_array(_ds.values[:], _ds.values, 33,sequence_stride=34, batch_size=111362)

In [22]:
import tensorflow as tf
for inputs, targets in X.take(1):
    print(tf.squeeze(inputs)[0].numpy(), '-->', tf.squeeze(targets)[0].numpy())
    print("Input shape:", inputs.numpy().shape)
    print("Target shape:", targets.numpy().shape)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 2.
 2. 0. 0. 0. 1. 1. 1. 3. 1.] --> 0.0
Input shape: (111362, 33, 1)
Target shape: (111362, 1)


 ## Definición de modelo

In [23]:
def build_model(lr):

  model = keras.Sequential([
    keras.layers.LSTM(units=8, batch_input_shape=(111362, 33, 1), stateful=True),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation='relu')
  ])

  model.compile(loss='mse',
                optimizer=Adam(learning_rate=lr),
                metrics=['accuracy','mean_squared_error'])

  return model

 ## Entrenamiento del modelo


### Instaciacion del modelo.

In [26]:
keras.utils.set_random_seed(0)
model = build_model(0.0001)

### entrenamiento normal

In [27]:
for epoc in range(10):
    model.fit(X, verbose=1, epochs=1, shuffle=False)
    model.reset_states()

1/1 [==============================] - 10s 10s/step - loss: 2.0766 - accuracy: 0.8475 - mean_squared_error: 2.0766


 ## Predicciones

In [28]:
from tensorflow.keras.utils import timeseries_dataset_from_array

_test = _ds.drop(ds.index[::34])
_test = timeseries_dataset_from_array(_test[:], None, 33, sequence_stride=33,batch_size=111362)

In [36]:
ds['ID'].unique().shape

(111362,)

In [38]:
preds = model.predict(_test).ravel()
preds = pd.DataFrame({ 'ID': ds['ID'].unique(), 'item_cnt_month': preds.astype(int)})
preds = pd.concat([preds, pd.DataFrame({'ID':test_zero, 'item_cnt_month':0})], ignore_index=True)
submission = preds.sort_values(by=['ID'])

1/1 [==============================] - 7s 7s/step


In [39]:
# creating csv file from dataframe
submission.to_csv('./sub_pfs.csv',index = False)